In [2]:
# import dependenices.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import requests
from citipy import citipy
from config import weather_api_key
from datetime import datetime

In [3]:
# Create 2,000 random latitudes and longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
coordinates = list(lat_lngs)
print(coordinates[0:3])

[(-50.12780562167458, -106.60841289481499), (-64.86514531599408, -19.868052396034585), (22.115727909100997, -131.74239992386606)]


In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude 
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # add to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count
len(cities)

735

In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=fad789c726d16e7e93753e4cef972869


In [7]:
r = requests.get(url + "&q=" + cities[13].replace(" ","+") ).json()
r

{'coord': {'lon': -68.3, 'lat': -54.8},
 'weather': [{'id': 801,
   'main': 'Clouds',
   'description': 'few clouds',
   'icon': '02d'}],
 'base': 'stations',
 'main': {'temp': 57.2,
  'feels_like': 33.19,
  'temp_min': 57.2,
  'temp_max': 57.2,
  'pressure': 1000,
  'humidity': 62},
 'visibility': 10000,
 'wind': {'speed': 40.29, 'deg': 230, 'gust': 55.23},
 'clouds': {'all': 20},
 'dt': 1612459020,
 'sys': {'type': 1,
  'id': 8303,
  'country': 'AR',
  'sunrise': 1612429433,
  'sunset': 1612485024},
 'timezone': -10800,
 'id': 3833367,
 'name': 'Ushuaia',
 'cod': 200}

In [8]:
# Create an empty list to hold the weather data.
city_data = []
skip_count = 0
# Loop through all the cities in the list.
for i, city in enumerate(cities):
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather['weather'][0]['description']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_description})

# If an error is experienced, skip the city.
    except:
        skip_count += 1
        continue
# print api summary notes.
print('{} cities skipped.'.format(skip_count)) 
print('data for {} cities retrieved '.format(len(city_data)))

48 cities skipped.
data for 687 cities retrieved 


In [9]:
len(city_data)

687

In [10]:
# Add the data to a list in a dict then at the DF
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Castro,BR,-24.7911,-50.0119,80.82,71,29,10.13,scattered clouds
1,Chuy,UY,-33.6971,-53.4616,73.76,61,90,20.38,overcast clouds
2,Lompoc,US,34.6391,-120.4579,54.00,66,1,10.11,clear sky
3,Viedma,AR,-40.8135,-62.9967,91.00,20,0,2.17,clear sky
4,Tiksi,RU,71.6872,128.8694,-24.77,86,98,3.42,overcast clouds
5,Belmonte,BR,-15.8631,-38.8828,84.20,74,20,10.36,few clouds
6,Hilo,US,19.7297,-155.0900,66.20,82,90,3.44,moderate rain
7,Beringovskiy,RU,63.0500,179.3167,-1.14,92,100,21.09,snow
8,Jamestown,US,42.0970,-79.2353,32.00,64,1,3.44,clear sky
9,Yulara,AU,-25.2406,130.9889,69.80,68,90,14.97,overcast clouds


In [11]:
# Create CSV file of DF
csv_weather_file = "WeatherPy_database.csv"
# Export CSV file 
city_data_df.to_csv(csv_weather_file, index_label="City_ID")